# Prototype Climate Impact Lab Interface

## Workspace Setup

Import some python modules

In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
import ClimateImpactLab as lab
reload(lab)

<module 'ClimateImpactLab' from 'ClimateImpactLab/__init__.pyc'>

## Connect to the API

Authenticate with the [prototype] API, and define how you want to use the system

In [2]:
api = lab.Client(
    username='mdelgado', 
    password='*********')

api.refresh_database()

In [3]:
api.commit()

## Retrieve formatted Climate Impact Lab variables

In [4]:
api.list_variables()

[u'GCP.Climate.Projection.GridQtrDeg.Precipitation',
 u'GCP.Climate.Projection.GridQtrDeg.Tas',
 u'GCP.Climate.Projection.GridQtrDeg.Tasmax',
 u'GCP.Climate.Projection.GridQtrDeg.Tasmin',
 u'GCP.Climate.Projection.IR.Precipitation',
 u'GCP.Climate.Projection.IR.Tas',
 u'GCP.Climate.Projection.IR.Tasmax',
 u'GCP.Climate.Projection.IR.Tasmin']

In [5]:
api.variables.GCP.Climate.Projection.GridQtrDeg

<ClimateImpactLab.api.variable._VariableGetter object : [u'Precipitation', u'Tas', u'Tasmax', u'Tasmin']>

In [6]:
api.list_dims()

[u'GCP.INDEX.Day',
 u'GCP.INDEX.Day365',
 u'GCP.INDEX.Gadm',
 u'GCP.INDEX.HistoricalPeriod',
 u'GCP.INDEX.IR',
 u'GCP.INDEX.LatDEG',
 u'GCP.INDEX.LatHalfDeg',
 u'GCP.INDEX.LatQtrDeg',
 u'GCP.INDEX.LatTwoDeg',
 u'GCP.INDEX.LonDEG',
 u'GCP.INDEX.LonHalfDeg',
 u'GCP.INDEX.LonQtrDeg',
 u'GCP.INDEX.LonTwoDeg',
 u'GCP.INDEX.Month',
 u'GCP.INDEX.ProjectionPeriods',
 u'GCP.INDEX.TempBinMortality',
 u'GCP.INDEX.Year']

In [7]:
api.list_scenarios()

[u'GCP.Scenarios.ClimateModels', u'GCP.Scenarios.RCPs', u'GCP_SCENARIOS_SSPs']

In [8]:
tasmin  = api.get_variable('GCP.Climate.Projection.GridQtrDeg.Tasmin')
tasmax  = api.get_variable('GCP.Climate.Projection.GridQtrDeg.Tasmax')

In [9]:
tasmin

<xarray.DataArray (GCP.INDEX.LatQtrDeg: 1, GCP.INDEX.LonQtrDeg: 1, GCP.INDEX.Day: 1)>
dask.array<xarray-..., shape=(1, 1, 1), dtype=float64, chunksize=(1, 1, 1)>
Coordinates:
  * GCP.INDEX.LatQtrDeg  (GCP.INDEX.LatQtrDeg) |S1 ''
  * GCP.INDEX.LonQtrDeg  (GCP.INDEX.LonQtrDeg) |S1 ''
  * GCP.INDEX.Day        (GCP.INDEX.Day) |S1 ''
Attributes:
    category: Climate.Projection
    latex: T
    additional_metadata: {u'version': u'1.0', u'driving_experiment': u'historical', u'resolution_id': u'0.25 degree', u'parent_experiment_id': u'historical', u'product': u'output', u'frequency': u'day', u'references': u'BCSD method: Thrasher et al., 2012, Hydrol. Earth Syst. Sci.,16, 3309-3314. Ref period obs: latest version of the Princeton Global Meteorological Forcings (http://hydrology.princeton.edu/data.php), based on Sheffield et al., 2006, J. Climate, 19 (13), 3088-3111. DOI: http://dx.doi.org/10.7292/W0MW2F2G...
    description: Gridded daily minimum near-surface air temperature
    author: Jiaca

In [10]:
tas = (tasmin + tasmax)/2

In [11]:
tasmin.derived

False

In [12]:
tas.derived

True

In [13]:
tasmin.display()

<IPython.core.display.Latex object>

In [17]:
tas.display()

<IPython.core.display.Latex object>

In [18]:
tas.attrs

OrderedDict([(u'gcp_id', 'GCP.Climate.Projection.GridQtrDeg.Tas2'),
             (u'name', 'Tas2')])

In [16]:
tas.attrs[u'gcp_id'] = 'GCP.Climate.Projection.GridQtrDeg.Tas2'
tas.attrs[u'name'] = 'Tas2'

In [19]:
all_the_tas = tas.sum(dim='GCP.INDEX.Day')

In [20]:
all_the_tas

<xarray.DataArray (GCP.INDEX.LatQtrDeg: 1, GCP.INDEX.LonQtrDeg: 1)>
dask.array<sum-agg..., shape=(1, 1), dtype=float64, chunksize=(1, 1)>
Coordinates:
  * GCP.INDEX.LatQtrDeg  (GCP.INDEX.LatQtrDeg) |S1 ''
  * GCP.INDEX.LonQtrDeg  (GCP.INDEX.LonQtrDeg) |S1 ''

In [21]:
all_the_tas.display()

<IPython.core.display.Latex object>

In [ ]:
api.database[tasmax.attrs['gcp_id']]

In [ ]:
api.publish(tas, latex='T')

## Do math with Impact Lab variables

In [ ]:
# From the NAS mortality presentation...

betahat = alpha+gamma1*avg_days_per_bin+gamma2*(gdp_per_capita.ln())+gamma3*(pop_density.ln())

In [ ]:
# define how your new variable should look
betahat.symbol = '\\hat{{\\beta}}'

In [ ]:
# show LaTeX for the math we just did
betahat.display()

In [ ]:
class Dim(object):
    def __init__(self, name, id):
        self.name = name
        self.id = id
    def __repr__(self):
        return self.name
    
tasmin.value.dims = (
    Dim('lat', 'GCP.Index.Latitude'), 
    Dim('lon', 'GCP.Index.Longitude'),
    Dim('time', 'GCP.Index.Days365'))


## Use functions from xArray and the rest of python

In [ ]:
ax = betahat.value.plot()
plt.show(ax)

## The next step in the calculation...

In [ ]:
# simplify the representation of betahat:
betahat.symbolic = '\\hat{{\\beta}}'

In [ ]:
# sum betahat*temp across bins
mortality = (betahat * temp).sum(dim='bins')

# declare a symbol for mortality
mortality.symbol = 'M'

In [ ]:
mortality.display()

Note that the dimensionality of mortality is less than beta and temp in the LaTeX and data:

In [ ]:
mortality.value

In [ ]:
ax = mortality.value.mean(dim='adm2').plot()
plt.show(ax)